# Цепочки и последовательности промптов

## Обзор

В этом уроке разбираются цепочки промптов (prompt chaining) и последовательное промптирование (sequencing) — приёмы, при которых выход одного промпта становится входом следующего.

## Мотивация

Чем сложнее задача, тем чаще её нужно разбивать на шаги. Цепочки промптов позволяют провести модель через серию связанных запросов — это даёт более структурированный и управляемый результат. Подход особенно полезен, когда задача требует нескольких этапов обработки или принятия решений.

## Ключевые компоненты

1. **Базовая цепочка промптов**: выход одного промпта подаётся на вход другого.
2. **Последовательное промптирование**: логическая последовательность промптов для многошагового анализа.
3. **Динамическая генерация промптов**: следующий промпт формируется на основе ответа модели.
4. **Обработка ошибок и валидация**: проверки внутри цепочки для повышения надёжности.

---

## Установка окружения

In [ ]:
import os
import re

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

from dotenv import load_dotenv
load_dotenv(os.path.join(os.path.dirname(os.getcwd()), ".env"))
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('BASE_URL')

llm = ChatOpenAI(model="gpt-4o-mini", base_url=base_url)

## Базовая цепочка промптов

Начнём с простого примера: первый промпт генерирует объяснение учебной темы для школьников, второй — сжимает его в одно ключевое предложение.

In [ ]:
# Шаблоны промптов
explain_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Объясни тему «{topic}» простым языком для школьника 7-го класса. 3–4 предложения."
)

summarize_prompt = PromptTemplate(
    input_variables=["explanation"],
    template="Сократи следующее объяснение до одного ключевого предложения:\n{explanation}"
)

# Цепочка промптов
def explain_and_summarize(topic):
    """Генерирует объяснение темы и сжимает его в одно предложение.

    Args:
        topic (str): Учебная тема.

    Returns:
        tuple: (объяснение, краткий итог).
    """
    explanation = (explain_prompt | llm).invoke({"topic": topic}).content
    summary = (summarize_prompt | llm).invoke({"explanation": explanation}).content
    return explanation, summary

# Тестируем цепочку
topic = "фотосинтез"
explanation, summary = explain_and_summarize(topic)
print(f"Объяснение:\n{explanation}\n\nКлючевое предложение:\n{summary}")

## Последовательное промптирование

Создадим более сложную последовательность промптов для многошагового анализа. Возьмём текст о методике преподавания и определим его основную тему, тон и ключевые выводы.

In [ ]:
# Шаблоны промптов для каждого шага анализа
theme_prompt = PromptTemplate(
    input_variables=["text"],
    template="Определи основную тему следующего текста:\n{text}"
)

tone_prompt = PromptTemplate(
    input_variables=["text"],
    template="Опиши общий тон следующего текста:\n{text}"
)

takeaway_prompt = PromptTemplate(
    input_variables=["text", "theme", "tone"],
    template="Учитывая текст с основной темой «{theme}» и тоном «{tone}», сформулируй ключевые выводы:\n{text}"
)

def analyze_text(text):
    """Многошаговый анализ текста: тема, тон, выводы.

    Args:
        text (str): Текст для анализа.

    Returns:
        dict: Словарь с темой, тоном и ключевыми выводами.
    """
    theme = (theme_prompt | llm).invoke({"text": text}).content
    tone = (tone_prompt | llm).invoke({"text": text}).content
    takeaways = (takeaway_prompt | llm).invoke({"text": text, "theme": theme, "tone": tone}).content
    return {"Тема": theme, "Тон": tone, "Выводы": takeaways}

# Тестируем последовательное промптирование
sample_text = """Современные исследования в области педагогики показывают, что активное обучение значительно 
эффективнее пассивного прослушивания лекций. Когда ученики решают задачи, обсуждают материал в группах 
и применяют знания на практике, уровень усвоения возрастает в разы. Однако переход к активным методам 
требует от учителя перестройки подхода: нужно больше времени на подготовку, умение управлять групповой 
динамикой и готовность отойти от привычного формата «учитель говорит — ученик слушает». Несмотря на 
сложности, школы, внедрившие активное обучение, фиксируют рост мотивации и успеваемости учащихся."""

analysis = analyze_text(sample_text)
for key, value in analysis.items():
    print(f"{key}: {value}\n")

## Динамическая генерация промптов

Создадим систему «вопрос — ответ», которая автоматически генерирует уточняющие вопросы на основе предыдущих ответов.

In [ ]:
# Шаблоны промптов
answer_prompt = PromptTemplate(
    input_variables=["question"],
    template="Ответь на вопрос кратко:\n{question}"
)

follow_up_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="На основе вопроса «{question}» и ответа «{answer}» сформулируй уточняющий вопрос по теме."
)

def dynamic_qa(initial_question, num_follow_ups=3):
    """Динамическая сессия «вопрос — ответ» с автоматическими уточняющими вопросами.

    Args:
        initial_question (str): Начальный вопрос.
        num_follow_ups (int): Количество уточняющих вопросов.

    Returns:
        list: Список словарей с вопросами и ответами.
    """
    qa_chain = []
    current_question = initial_question

    for i in range(num_follow_ups + 1):  # +1 для начального вопроса
        answer = (answer_prompt | llm).invoke({"question": current_question}).content
        qa_chain.append({"question": current_question, "answer": answer})
        
        if i < num_follow_ups:  # уточняющий вопрос для всех итераций, кроме последней
            current_question = (follow_up_prompt | llm).invoke({"question": current_question, "answer": answer}).content

    return qa_chain

# Тестируем динамическую генерацию
initial_question = "Какие методы помогают повысить мотивацию учеников к учёбе?"
qa_session = dynamic_qa(initial_question)

for i, qa in enumerate(qa_session):
    print(f"В{i+1}: {qa['question']}")
    print(f"О{i+1}: {qa['answer']}\n")

## Обработка ошибок и валидация

В этой части добавляем обработку ошибок и проверку результатов внутри цепочки промптов, чтобы сделать её надёжнее.

In [ ]:
# Шаблоны промптов
generate_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Назови 4-значное число, связанное с темой: {topic}. Ответь ТОЛЬКО числом, без пояснений."
)

validate_prompt = PromptTemplate(
    input_variables=["number", "topic"],
    template="Число {number} действительно связано с темой «{topic}»? Ответь «Да» или «Нет» и объясни почему."
)

def extract_number(text):
    """Извлекает 4-значное число из текста.

    Args:
        text (str): Текст для извлечения числа.

    Returns:
        str or None: 4-значное число или None, если не найдено.
    """
    match = re.search(r'\b\d{4}\b', text)
    return match.group() if match else None

def robust_number_generation(topic, max_attempts=3):
    """Генерация числа, связанного с темой, с валидацией и обработкой ошибок.

    Args:
        topic (str): Тема для генерации числа.
        max_attempts (int): Максимальное количество попыток.

    Returns:
        str: Валидированное 4-значное число или сообщение об ошибке.
    """
    for attempt in range(max_attempts):
        try:
            response = (generate_prompt | llm).invoke({"topic": topic}).content
            number = extract_number(response)
            
            if not number:
                raise ValueError(f"Не удалось извлечь 4-значное число из ответа: {response}")
            
            # Валидация связи числа с темой
            validation = (validate_prompt | llm).invoke({"number": number, "topic": topic}).content
            if validation.lower().startswith("да"):
                return number
            else:
                print(f"Попытка {attempt + 1}: число {number} не прошло валидацию. Причина: {validation}")
        except Exception as e:
            print(f"Попытка {attempt + 1} не удалась: {str(e)}")
    
    return "Не удалось сгенерировать валидное число после нескольких попыток."

# Тестируем генерацию с валидацией
topic = "Великая Отечественная война"
result = robust_number_generation(topic)
print(f"Результат для темы «{topic}»: {result}")